*** Unsupervised Text Feature Extraction ***

History:  
- v1: first version, 2017-09-26
- v2: update, 2017-10-09
  - 先用結巴斷詞
      - 中文以詞為單位, 而不是用字
      - 英文以單字為單位, 而不是用char
  - 結合N*N單字矩陣時, 兩個英文字中間加一個空白, 解決兩個英文單字結合時, 無法計算分數的問題
  - 儲存分析每一個iteration的關鍵字變化
  - 用簡總的文章再做一次
  - 用多篇文章找關鍵字
  

- TODO:     
  - 確認之前有哪一篇可以找出常用詞彙(應該是用簡總會議全部文章)

  
   

In [1]:
import os
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import jieba
import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import re
import ast

import random

import logging
import html_template as ht
import collections
from collections import OrderedDict
import itertools
import math

from itertools import combinations
from itertools import permutations
import argparse
    
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

%matplotlib inline
%load_ext autoreload
%autoreload 2


pd.set_option('display.max_columns', None)
punct = u'''\n +-％%:!),.:;?]}¢'"、。〉》」』】〕〗〞︰|︱︳丨﹐､﹒﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠々‖•·ˇˉ―′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻︽︿﹁﹃﹙﹛﹝（｛“‘—_…~/#><'''
jieba.set_dictionary('dict.txt.big')
jieba.load_userdict('userdict.txt')

Building prefix dict from /Users/jayhsu/work/github/textmining/dict.txt.big ...
Loading model from cache /var/folders/yf/hq7ghg4j3ksb8k34wyh8vk2m0000gn/T/jieba.ub36e993abda9bbf53d1f5b38e3ae9b44.cache
Loading model cost 2.384 seconds.
Prefix dict has been built succesfully.


In [2]:
data_cleanregex = ['\[SS_\w+\]','\(adsbygoogle.+\)\;']
data_deleteregex_ = [r'.*\.srt$']

def data_clean(dirty_str):
    tmp_str = dirty_str
    for pat in data_cleanregex:
        tmp_str = re.sub(pat, '', tmp_str)
    return tmp_str

data_deleteregex = list(map(lambda x: re.compile(x), data_deleteregex_))
def data_delete(dirty_str):
    tmp_str = dirty_str
    for pattern in data_deleteregex:
        #pattern = re.compile(pat)  
        match1 = pattern.match(dirty_str)
        if match1:
            return True
    return False   
def loaddocs(folder, topic):
    files = os.listdir('{f}/{w}'.format(f=folder, w=topic))
    docs=[]
    fileName=[]
    
    for fidx, fname in enumerate(files):
        if fname.startswith('.') or fname.endswith('.flag') :
            continue

        file = open('{fd}/{w}/{f}'.format(fd=folder, w=topic, f=fname), 'r') 
        doc =[] 
        for line in file.readlines():                          
            line = line.strip()
            if not len(line):                               
                continue
            if data_delete(line):
                continue
                
            line = data_clean(line)
            
            doc.append(line) 
        if len(doc)==0:
            continue
        docs.append('\n'.join(doc))
        fileName.append(str(fidx)+'_'+fname)
    dl_df = pd.DataFrame({'fname':fileName, 'doc':docs})
    return dl_df

In [3]:
def remove_similar(powerterm):
    remove_list=[]
    for tup in powerterm:
        tmp2 = powerterm.copy()
        tmp2.remove(tup)
        tmp2 = dict(tmp2)
        result = [(key, value) for key, value in tmp2.items() if tup[0] in key]
        if len(result)==0:
            continue
        result = sorted(result, key=lambda tup: tup[1], reverse=True)
        if tup[1] < result[0][1]:
            remove_list.append(tup)

    _ = list(map(lambda tup:powerterm.remove(tup), remove_list))    

''' 
Example Case: 
tmplist = [('aa',1),('bb',1),('cc',1),('aa',1),]
remove_duplicate(tmplist)
display(tmplist)
'''

def is_chinese(uchar):         
    if '\u4e00' <= uchar<='\u9fff':
        return True
    else:
        return False
    

def is_number(uchar):
    if '\u0030' <= uchar<='\u0039':
        return True
    else:
        return False

def is_alphabet(uchar):         
    if ('\u0041' <= uchar<='\u005a') or ('\u0061' <= uchar<='\u007a'):
        return True
    else:
        return False

In [149]:
len(nature_chunk)

109

In [4]:
#計算分數的function
def NC_Scoring(tmp, nature_chunk):
    score =0
    subscore_fb=subscore_fl=subscore_fr=subscore_fn=0
    for chunk in nature_chunk:
        if tmp == chunk:
            subscore_fb = subscore_fb+1
        if chunk.startswith(tmp):
            subscore_fl = subscore_fl+1
        if chunk.endswith(tmp):
            subscore_fr = subscore_fr+1
        if tmp in chunk:
            subscore_fn = subscore_fn+1
    score = 0.5*subscore_fb + 0.2*(subscore_fl+subscore_fr)+0.1*subscore_fn
    score = len(tmp) * score
    return float('%.3f'% score)

#NC_Scoring('TITAN X')

In [5]:
def __getPowerTerm__(powerterm, paragraph, nature_chunk):
    #1. 斷字
    corpora_sentence = []
    for p in paragraph:
        term_known = list(filter(lambda t: t[0] in p, powerterm))
        term_known = list(map(lambda tup: tup[0], term_known))
        for w in powerterm:
            p = p.replace(w[0], '')
        p_cut = list(p)
        p_cut = list(filter(lambda x: x not in punct, p_cut))
        corpora_sentence.append(p_cut+term_known)
    
    #2. LDA
    sen_dictionary = gensim.corpora.Dictionary(corpora_sentence)  
    sen_corpus = [sen_dictionary.doc2bow(text) for text in corpora_sentence]  

    K = 5
    lda = gensim.models.ldamodel.LdaModel(corpus=sen_corpus, id2word=sen_dictionary, num_topics=K, update_every=0, passes=1)  
    corpus_lda = lda[sen_corpus] 
    wordTopic = lda.print_topics(num_topics=-1, num_words=15)
    wordTopic_pure = (list(map(lambda tup: (list(map(lambda tup: tup.split('*')[1].replace('"','').strip(),tup[1].split('+')))   ),wordTopic)))

    #3. 產生NxN term candidate, scoring排序, 取前5%為power term
    wordTopic_NN = list(map(lambda x: list(map(lambda tup: tup[0]+tup[1], list(permutations(x, 2)))) , wordTopic_pure))
    powerterm_candidate = set(sum(wordTopic_NN,[]))
    powerterm = list(map(lambda term: (term, NC_Scoring(term, nature_chunk)), powerterm_candidate))
    powerterm = list(filter(lambda tup: tup[1]>0,powerterm))
    powerterm = sorted(powerterm, key=lambda tup: tup[1], reverse=True)
    #powerterm = powerterm[: int(len(powerterm)*0.1)]
    #powerterm = list(map(lambda tup: tup[0], powerterm))
    return powerterm


In [279]:
def getPowerTerm(powerterm, paragraph, usejieba=False, num_words=15):
    #1. 斷字
    corpora_sentence = []
    for p in paragraph:
        term_known = list(filter(lambda t: t[0] in p, powerterm))
        term_known = list(map(lambda tup: tup[0], term_known))
        for w in term_known:
            p = p.replace(w, '')
        if usejieba:
            p_cut = jieba.cut(p)
            p_cut = list(p_cut)
        else:
            p_cut = list(p)
        p_cut = list(filter(lambda x: x not in punct, p_cut))
        corpora_sentence.append(p_cut+term_known)
    
    #2. LDA
    sen_dictionary = gensim.corpora.Dictionary(corpora_sentence)  
    sen_corpus = [sen_dictionary.doc2bow(text) for text in corpora_sentence]  

    K = 5
    lda = gensim.models.ldamodel.LdaModel(corpus=sen_corpus, id2word=sen_dictionary, num_topics=K, update_every=0, passes=1)  
    corpus_lda = lda[sen_corpus] 
    wordTopic = lda.print_topics(num_topics=-1, num_words=num_words)
    wordTopic_pure = (list(map(lambda tup: (list(map(lambda tup: tup.split('*')[1].replace('"','').strip(),tup[1].split('+')))   ),wordTopic)))
    wordTopic_pure = list(map(lambda x: list(filter(lambda y: len(y.strip())!=0, x)),wordTopic_pure))
    print(wordTopic_pure)
    #3. 產生NxN term candidate, scoring排序, 取前5%為power term
    #wordTopic_NN = list(map(lambda x: list(map(lambda tup: tup[0]+tup[1], list(permutations(x, 2)))) , wordTopic_pure))
    wordTopic_NN = list(map(lambda x: list(map(lambda tup: ' '.join([tup[0],tup[1]]) if is_alphabet(tup[0]) and is_alphabet(tup[1]) else ''.join([tup[0],tup[1]]), list(permutations(x, 2)))) , wordTopic_pure))
    
    #powerterm_candidate = set(sum(wordTopic_NN,[]))
    #powerterm = list(map(lambda term: (term, NC_Scoring(term, nature_chunk)), powerterm_candidate))
    #powerterm = list(filter(lambda tup: tup[1]>0,powerterm))
    #powerterm = sorted(powerterm, key=lambda tup: tup[1], reverse=True)
    #powerterm = powerterm[: int(len(powerterm)*0.1)]
    #powerterm = list(map(lambda tup: tup[0], powerterm))
    
    powerterm_list = []
    for words in wordTopic_NN:
        powerterm_candidate = set(words)
        powerterm = list(map(lambda term: (term, NC_Scoring(term, nature_chunk)), powerterm_candidate))
        powerterm = list(filter(lambda tup: tup[1]>0,powerterm))
        powerterm = sorted(powerterm, key=lambda tup: tup[1], reverse=True)
        powerterm_list.append(powerterm)
    return powerterm_list


In [228]:
nature_chunk=[]
def main(doc, itera = 20, linebreaknum=1, num_words=15):
    global nature_chunk
    paragraph_tmp = list(filter(lambda x: x != '...', doc.split('\n')))
    paragraph =[]
    for idx, item in enumerate (paragraph_tmp):
        if idx%linebreaknum ==0:
            paragraph.append('。'.join(paragraph_tmp[idx:idx+linebreaknum]))
    print(len(paragraph))
    print(paragraph)
    #產生nature_chunk
    nature_chunk = []
    for p in paragraph:
        chunk = ''.join((char if char.isalpha() or char.isdigit() or char.isspace() else '|') for char in p).strip().split('|')
        chunk = list(map(lambda x: x.strip(), chunk))
        nature_chunk = nature_chunk+chunk
    nature_chunk = list(filter(lambda a: a != '', nature_chunk))
    print(len(nature_chunk))
    #print(nature_chunk)
    powerterm = []
    powerterm_eachiter=[]
    powerterm_list=[]
    powerterm_list_eachiter=[]
    powerterm_word=[]
    for i in range(itera):
        powerterm_list= getPowerTerm(powerterm, paragraph,usejieba=True, num_words=num_words)
        powerterm_new = list(filter(lambda x: x[0] not in powerterm_word, sum(powerterm_list,[])))
        powerterm = powerterm + list(set(powerterm_new))
        powerterm = sorted(powerterm, key=lambda tup: tup[1], reverse=True)
        remove_similar(powerterm)
        powerterm_word = list(map(lambda tup: tup[0], powerterm))
        powerterm_eachiter.append(powerterm)
        powerterm_list_eachiter.append(powerterm_list)
    return powerterm_list_eachiter, powerterm_eachiter



In [263]:
def main2(doc, itera = 20, linebreaknum=1,num_words=15):
    global nature_chunk
    #paragraph_tmp = list(filter(lambda x: x != '...', doc.split('\n')))
    #paragraph =[]
    #for idx, item in enumerate (paragraph_tmp):
    #    if idx%linebreaknum ==0:
    #        paragraph.append('。'.join(paragraph_tmp[idx:idx+linebreaknum]))
    paragraph = list(map(lambda d: '。'.join(list(filter(lambda x: x != '...', d.split('\n')))), doc))
    print(len(paragraph))
    #print(paragraph)
    #產生nature_chunk
    nature_chunk = []
    for p in paragraph:
        chunk = ''.join((char if char.isalpha() or char.isdigit() or char.isspace() else '|') for char in p).strip().split('|')
        chunk = list(map(lambda x: x.strip(), chunk))
        nature_chunk = nature_chunk+chunk
    nature_chunk = list(filter(lambda a: a != '', nature_chunk))
    print('\n----------------------\n')
    print(len(nature_chunk))
    #print(nature_chunk)
    powerterm = []
    powerterm_eachiter=[]
    powerterm_list=[]
    powerterm_list_eachiter=[]
    powerterm_word=[]
    for i in range(itera):
        print('itera:',i)
        powerterm_list= getPowerTerm(powerterm, paragraph,usejieba=True, num_words=num_words)
        powerterm_new = list(filter(lambda x: x[0] not in powerterm_word, sum(powerterm_list,[])))
        powerterm = powerterm + list(set(powerterm_new))
        powerterm = sorted(powerterm, key=lambda tup: tup[1], reverse=True)
        remove_similar(powerterm)
        powerterm_word = list(map(lambda tup: tup[0], powerterm))
        powerterm_eachiter.append(powerterm)
        powerterm_list_eachiter.append(powerterm_list)
    return powerterm_list_eachiter, powerterm_eachiter

***
**Deep Learning Doc **

In [186]:
# CONFIGs
parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', type=str, default='text/techdoc',)
parser.add_argument('--topic', type=str, default='深度學習',)  #'MARK'
parser.add_argument('--docid', type=str, default=13,)  
parser.add_argument('--iter', type=str, default=50,)  
args,unparsed = parser.parse_known_args()

dl_df = loaddocs(folder = args.data_dir, topic=args.topic)

In [187]:
%%time
doc = dl_df['doc'][args.docid]
powerterm_list_eachiter, powerterm_eachiter= main(doc, args.iter,linebreaknum=1)

26
109
CPU times: user 13.7 s, sys: 87.4 ms, total: 13.7 s
Wall time: 13.9 s


In [188]:
for i in range(args.iter):
    print('\n\niter ',i)
    topic_powerterm = pd.DataFrame(powerterm_list_eachiter[i])
    display(topic_powerterm.transpose().head()) 
    top_powerterm = pd.DataFrame(powerterm_eachiter[i])
    display(top_powerterm[:5+i])



iter  0


,0,1,2,3,4
0,"(TITAN X, 10.5)","(TITAN X, 10.5)","(DIGITS DevBox, 35.1)","(用於深度學習, 1.8)","(訓練系統, 4.0)"
1,"(DIGITS深度學習, 7.0)","(DIGITS深度學習, 7.0)","(DIGITS深度學習, 7.0)","(的深度學習, 1.5)","(NVIDIA深度學習, 3.0)"
2,"(GeForce GTX, 6.6)","(使用DIGITS, 4.8)","(使用DIGITS, 4.8)","(深度學習的, 1.0)","(DevBox的, 2.1)"
3,"(使用DIGITS, 4.8)","(訓練系統, 4.0)","(NVIDIA深度學習, 3.0)","(的系統, 0.9)","(的深度學習, 1.5)"
4,"(訓練系統, 4.0)","(X GPU, 3.0)","(DevBox的, 2.1)","(系統中, 0.3)","(GPU訓練, 1.5)"


,0,1
0,DIGITS DevBox,35.1
1,TITAN X,10.5
2,DIGITS深度學習,7.0
3,GeForce GTX,6.6
4,使用DIGITS,4.8




iter  1


,0,1,2,3,4
0,"(TITAN X GPU, 6.6)","(深度學習GPU訓練系統, 6.6)","(GeForce GTX TITAN X, 13.3)","(TITAN X GPU, 6.6)","(GeForce GTX TITAN X, 13.3)"
1,"(使用DIGITS DevBox, 4.5)","(GPU訓練系統, 4.2)","(TITAN X GPU, 6.6)","(採用TITAN X, 2.7)","(NVIDIA GeForce GTX, 5.4)"
2,"(的DIGITS DevBox, 4.2)","(DIGITS深度學習 GPU, 4.2)","(GPU訓練系統, 4.2)","(的TITAN X, 2.4)","(的DIGITS DevBox, 4.2)"
3,"(的TITAN X, 2.4)","(數據科學家和, 1.2)","(的DIGITS DevBox, 4.2)","(採用NVIDIA, 2.4)","(NVIDIA深度學習, 3.0)"
4,"(TITAN X在, 2.4)","(了數據科學家, 0.6)","(全球最快, 2.4)","(TITAN X的, 2.4)","(的TITAN X, 2.4)"


...

,0,1
0,DIGITS DevBox,35.1
1,現已上市的GeForce GTX TITAN X售價為7999元人民幣,35.0
2,在舊金山舉辦的遊戲開發者大會上讓各位先睹為快了TITAN X的身影,33.0
3,NVIDIA GeForce GTX TITAN X,26.0
4,對在座的四千名與會嘉賓展示三項將推動深度學習的新技術,26.0
5,使用深度神經網絡來訓練電腦教自己如何分類和識別物體,25.0
6,可為最嚴苛的深度學習研究工作提供高效率的性能表現,24.0
7,可應用在從先進醫藥研究到全自動駕駛汽車的多元領域,24.0
8,GeForce GTX TITAN X的另一面,23.0
9,訓練和驗證圖像分類深度神經網絡的多合一圖形系統,23.0




iter  49


,0,1,2,3,4
0,"(直觀的用戶介面和強大的工作流程管理能力, 5.7)","(總裁兼執行長黃仁勛先生在GPU技術大會的開幕主題演講活動上, 29.0)","(的DIGITS DevBox, 4.2)",None,"(對工作進行微調, 2.1)"
1,"(得益於其, 1.2)","(X GPU, 3.0)",None,None,"(廣泛使用, 1.2)"
2,"(其直觀, 0.3)",None,None,None,"(數據科學家和研究人員在, 1.1)"


,0,1
0,DIGITS DevBox,35.1
1,現已上市的GeForce GTX TITAN X售價為7999元人民幣,35.0
2,在舊金山舉辦的遊戲開發者大會上讓各位先睹為快了TITAN X的身影,33.0
3,總裁兼執行長黃仁勛先生在GPU技術大會的開幕主題演講活動上,29.0
4,NVIDIA GeForce GTX TITAN X,26.0
5,對在座的四千名與會嘉賓展示三項將推動深度學習的新技術,26.0
6,使用深度神經網絡來訓練電腦教自己如何分類和識別物體,25.0
7,可為最嚴苛的深度學習研究工作提供高效率的性能表現,24.0
8,可應用在從先進醫藥研究到全自動駕駛汽車的多元領域,24.0
9,GeForce GTX TITAN X的另一面,23.0


CPU times: user 1.88 s, sys: 429 ms, total: 2.31 s
Wall time: 2.72 s


*** 
**簡總Meeting Minutes **

In [227]:
# CONFIGs
parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', type=str, default='text/techdoc',)
parser.add_argument('--topic', type=str, default='MARK',)  #'MARK'
parser.add_argument('--docid', type=str, default=1,)  
parser.add_argument('--iter', type=str, default=20,)  
args,unparsed = parser.parse_known_args()

dl_df = loaddocs(folder = args.data_dir, topic=args.topic)

In [ ]:
%%time
doc = dl_df['doc'][args.docid]
powerterm_list_eachiter, powerterm_eachiter= main(doc, args.iter, linebreaknum=30, num_words=30)

In [243]:
n=30
print('all powerterm')
display(pd.DataFrame(powerterm_eachiter[n][:20]))
print('by topic')
topic_powerterm = pd.DataFrame(powerterm_list_eachiter[n])
display(topic_powerterm.transpose())

#powerterms
'''
for i in range(args.iter):
    print('\n\niter ',i)
    topic_powerterm = pd.DataFrame(powerterm_list_eachiter[i])
    display(topic_powerterm.transpose().head()) 
    top_powerterm = pd.DataFrame(powerterm_eachiter[i])
    display(top_powerterm[:5+i])
'''

all powerterm


,0,1
0,的,1760.9
1,深度學習,1723.2
2,網絡,746.6
3,1,692.7
4,n,641.7
5,2,639.6
6,是,587.9
7,在,556.9
8,我們,490.2
9,3,441.8


by topic


,0,1,2,3,4
0,"(卷積神經, 152.8)","(學習模型, 79.6)","(用於, 83.2)","(卷積神經, 152.8)","(卷積神經, 152.8)"
1,"(學習模型, 79.6)","(訓練數據, 68.4)","(學習模型, 79.6)","(學習模型, 79.6)","(學習模型, 79.6)"
2,"(訓練數據, 68.4)","(層神經, 51.6)","(訓練數據, 68.4)","(訓練數據, 68.4)","(訓練數據, 68.4)"
3,"(學習技術, 67.6)","(學習方法, 47.2)","(學習技術, 67.6)","(學習方法, 47.2)","(學習技術, 67.6)"
4,"(你可以, 63.6)","(可以通過, 44.4)","(層神經, 51.6)","(可以通過, 44.4)","(你可以, 63.6)"
5,"(就可以, 39.0)","(輸入層, 42.9)","(輸出層, 47.7)","(輸入數據, 42.4)","(你可, 50.0)"
6,"(人工智慧技術, 22.2)","(輸入數據, 42.4)","(學習方法, 47.2)","(學習領域, 42.4)","(學習方法, 47.2)"
7,"(它可以, 21.3)","(訓練方法, 41.2)","(可以通過, 44.4)","(訓練方法, 41.2)","(訓練方法, 41.2)"
8,"(訓練參數, 18.4)","(就可以, 39.0)","(輸入層, 42.9)","(40, 39.2)","(就可以, 39.0)"
9,"(你需要, 18.3)","(層模型, 31.8)","(輸入數據, 42.4)","(就可以, 39.0)","(特徵學習, 34.0)"


"\nfor i in range(args.iter):\n    print('\n\niter ',i)\n    topic_powerterm = pd.DataFrame(powerterm_list_eachiter[i])\n    display(topic_powerterm.transpose().head()) \n    top_powerterm = pd.DataFrame(powerterm_eachiter[i])\n    display(top_powerterm[:5+i])\n"

***
** All Doc **

In [283]:
# CONFIGs
parser = argparse.ArgumentParser()
parser.add_argument('--data_dir', type=str, default='text/techdoc',)
parser.add_argument('--topic', type=str, default='深度學習',)  #'MARK'
parser.add_argument('--docid', type=str, default=1,)  
parser.add_argument('--iter', type=str, default=100,)  
args,unparsed = parser.parse_known_args()

dl_df = loaddocs(folder = args.data_dir, topic=args.topic)

In [284]:
%%time
doc = dl_df['doc'].tolist()
powerterm_list_eachiter, powerterm_eachiter= main2(doc, args.iter, linebreaknum=1, num_words=30)

101

----------------------

43694
itera: 0
[['的', '是', '在', '和', '學習', '了', '深度學習', '一個', '中', '訓練', '可以', '有', '神經網絡', '模型', '數據', '特徵', '輸入', '我們', '上', '網絡', '也', '層', '對', '為', '算法', '就', '1', '機器', '使用', '進行'], ['的', '和', '深度學習', '是', '在', '了', '一個', '學習', '數據', '中', '可以', '我們', '也', '訓練', '模型', '上', '神經網絡', '有', '機器', '對', '人工智慧', '到', '為', 'io', '技術', '與', '都', '使用', '網絡', '需要'], ['的', '是', '在', '深度學習', '了', '和', '一個', '可以', '神經網絡', '我們', '中', '模型', '學習', '訓練', '數據', '也', '機器', '有', '算法', '上', '為', '你', '網絡', '到', '對', '都', '需要', '特徵', '它', '計算'], ['的', '在', '是', '深度學習', '了', '和', '一個', '模型', '中', '學習', '數據', '訓練', '可以', '也', 'io', '神經網絡', '我們', '網絡', '有', '都', '你', '為', '特徵', '卷積', '到', '上', '使用', '機器', '會', '這個'], ['的', '在', '了', '和', '是', '一個', '深度學習', '學習', '模型', '中', '神經網絡', '數據', '有', '可以', '網絡', '訓練', '為', '我們', '使用', '算法', '上', 'GPU', '也', '機器', '需要', '對', '都', '特徵', '進行', '到']]
itera: 1
[['的', '在', '和', '了', '是', '圖像', '中', '深度學習', '模型', '更', '一個', '通過', '可以', '與', '我們

itera: 10
[['的', '和', '了', '在', '是', '一個', '中', '上', '可以', '為', 'io', '層', '與', '1', '就', '我們', '使用', '網絡', '學習', 'GPU', '等', 'n', '訓練', '也', '更', '0', '3', '算法', '來', '而'], ['的', '和', '在', '是', 'io', '了', '中', '一個', '1', '可以', 'n', '模型', '也', '我們', '學習', '訓練', '=', '數據', '有', '2', '到', '網絡', '上', '3', '來', '都', '為', '與', '對', '而'], ['的', '和', '了', '在', '是', '中', 'io', '為', '上', '我們', '模型', '深度', '可以', '一個', 'n', '進行', '學習', '其', '有', '1', '數據', '到', '算法', '也', '生成', '通過', '輸入', '網絡', '對', '用'], ['的', '和', '在', '是', '了', '中', '上', 'io', '到', '一個', '我們', '你', '1', '學習', '=', '為', '數據', '進行', '與', '可以', '也', '有', '模型', '年', '等', '使用', '2', '這', '新', '對'], ['的', '在', '是', '和', '了', '=', 'io', '學習', '一個', '上', '我們', '中', '為', '也', '模型', '訓練', '數據', '與', '有', 'n', '對', '算法', '個', '可以', '將', '2', '層', '等', '1', '使用']]
itera: 11
[['的', '在', '和', '了', 'io', '是', '1', '中', '=', '一個', 'n', '為', '模型', '可以', '訓練', '上', '2', '我們', '與', '技術', '等', '3', '使用', '數據', '神經網絡', '需要', '系統', '學習', '算法', '這些

itera: 20
[['的', '在', '和', '了', '個', 'io', '是', '中', '可以', '我們', '一個', 'n', '與', '1', '2', '學習', '通過', '訓練', '網絡', '=', '3', '上', '為', '模型', '數據', '從', '使用', '到', '圖像', '人類'], ['的', '在', '和', '是', '了', '=', '中', '個', '上', '一個', '我們', '模型', 'TensorFlow', '為', '可以', '3', 'x', '與', 'io', '學習', 'n', '有', '也', '等', '網絡', '從', '2', '到', '以及', '將'], ['的', '和', '了', '在', '是', 'io', '個', '1', '可以', '也', '中', '我們', '模型', '特徵', '算法', '學習', '數據', '層', '訓練', '一個', '輸入', '網絡', '表示', 'GPU', '2', '上', '為', 'n', '神經網絡', '就'], ['的', '和', '在', '了', '是', '個', '中', 'io', '一個', '為', '學習', '上', '=', '1', '可以', '數據', '到', 'n', '與', '訓練', '我們', '有', '進行', '也', '對', '使用', 'GPU', '年', '更', '大'], ['的', '在', '和', '是', '了', 'io', '1', '個', '中', '=', '上', 'n', '一個', '學習', '也', '使用', '為', '網絡', '我們', '算法', '對', '進行', '層', '到', '2', '可以', '有', '最', '問題', '年']]
itera: 21
[['的', '和', '在', '了', '是', '個', 'io', '中', '一個', '為', '我們', '學習', '1', '到', '上', '可以', '進行', 'n', '有', '=', '更', '輸入', '網絡', '也', '算法', '訓練', '你', '3'

itera: 30
[['的', '和', '在', '了', '是', '個', '=', '中', 'io', '一個', '1', '也', '為', '與', '可以', 'n', '學習', '訓練', '2', '3', '上', '我們', '來', '有', '進行', '就', '網絡', '輸入', '到', '層'], ['的', '在', '是', '和', '了', '個', '中', '一個', 'io', '我們', '1', '模型', '進行', '學習', '為', 'n', '上', '有', '訓練', '層', '到', '來', '數據', '可以', '網絡', '2', '我', '使用', '與', '問題'], ['的', '在', '了', '和', '是', '個', 'io', '可以', '中', '=', '上', '為', '我們', '1', '而', '到', '等', '對', 'n', '學習', '並', '從', '2', '就', '與', '也', '進行', '年', '會', '以'], ['的', '在', '和', 'io', '是', '中', '了', '個', '上', '1', '一個', 'n', '學習', '可以', '我們', '模型', '對', '網絡', '使用', '=', '也', '為', '訓練', 'GPU', '層', '算法', '有', '3', '輸入', '到'], ['的', '和', '在', '是', '了', '個', 'io', '可以', '我們', '與', '學習', '中', '=', '數據', '為', 'n', '你', '2', '訓練', '一個', 'GPU', '1', '而', '模型', '3', '由', '算法', '也', '有', '特徵']]
itera: 31
[['的', '和', '在', '了', '是', '個', 'io', '中', '學習', '上', '我們', 'n', '可以', '網絡', '進行', '一個', '訓練', '1', '到', '使用', '對', '2', '也', '為', '下', '層', 'GPU', '而', '特徵', '模型'], ['

itera: 40
[['的', '在', '和', '是', '了', '個', 'io', '中', 'n', '1', '學習', '=', '上', '3', '一個', '我們', '與', '訓練', '算法', '為', '網絡', '模型', '有', '也', '來', '方面', '它', '可以', '對', '會'], ['的', '在', '和', '了', '是', '中', '=', 'io', '一個', '上', '個', '1', '可以', '為', '與', '也', '2', 'n', 'GPU', '使用', '而', '就', '訓練', '學習', '對', '我們', '需要', '都', '模型', '有'], ['的', '和', '在', '是', '了', 'io', '個', '中', 'n', '我們', '上', '對', '一個', '與', '學習', '可以', '從', '模型', '為', '會', '使用', '系統', '通過', '到', '3', '這些', '他', '於', '都', '函數'], ['的', '和', '在', '是', '了', '個', '1', 'io', '我們', '學習', '=', '訓練', '中', '一個', '為', '可以', 'GPU', '與', '2', '也', '3', '層', 'n', '上', '使用', '有', '等', '來', '需要', '函數'], ['的', '在', '和', '了', '是', '個', 'io', '中', '可以', '1', '數據', '我們', '模型', '一個', '網絡', '為', '2', '上', '也', '到', '學習', 'n', '進行', '而', '層', '輸入', '=', '對', '訓練', '特徵']]
itera: 41
[['的', '在', '和', '了', '是', 'io', '個', '中', '=', 'n', '1', '學習', '一個', '我們', '可以', '訓練', '也', '而', '上', '到', '網絡', '使用', '與', '算法', '模型', '從', '為', '有', '年', '2'], [

...

itera: 60
[['的', '和', '是', '在', '了', '個', '=', '1', '我們', 'io', '一個', '也', '中', '2', 'n', '進行', '為', '學習', '對', '網絡', '訓練', '可以', '層', '使用', '有', '模型', '來', '從', '3', '需要'], ['的', '和', '在', '了', '是', '上', '個', '中', '可以', '我們', '一個', 'io', '訓練', '1', '學習', '也', '有', '使用', 'n', '來', '層', '=', '與', '進行', '而', '數據', '以', '2', '為', '輸入'], ['的', '在', 'io', '和', '了', '是', '中', '個', 'n', '為', '3', '上', '訓練', '與', '學習', '一個', '我們', '模型', '可以', '網絡', '=', '1', '你', 'GPU', '2', '通過', '年', '對', '就', '需要'], ['的', '在', '和', '了', '是', 'io', '中', '個', '1', '為', '學習', '與', '我們', '一個', 'n', '算法', '上', '也', '=', '網絡', '對', '可以', '模型', '而', '等', '層', '數據', '年', '2', '都'], ['的', '在', '和', '了', '是', 'io', '中', '個', '=', '可以', '學習', '一個', '到', '模型', '1', '為', '進行', '2', '地', '與', 'n', '你', '來', '上', '需要', '而', '3', '年', '我們', '更']]
itera: 61
[['的', '在', '和', '是', '了', '中', '1', '個', '訓練', '上', 'io', '一個', '也', '可以', '為', '學習', '我們', '函數', '使用', '有', '進行', '3', '而', '算法', '需要', '年', '層', '圖像', '等', '模型'], ['的

itera: 70
[['的', '和', '了', '在', '=', '是', '1', '個', '中', '3', '可以', '上', 'n', '學習', 'io', '模型', 'x', '一個', '我們', '與', '為', '網絡', 'Deep', 'GPU', '對', '2', '也', '0', '技術'], ['的', '在', '和', '是', '了', 'io', '中', '個', '我們', '為', '1', '一個', '訓練', '算法', '可以', '2', '層', 'n', 'GPU', '等', '學習', '年', '而', '與', '網絡', '有', '也', '來', '就', '需要'], ['的', '在', '和', '了', '是', 'io', '一個', '中', '個', '=', '也', 'n', '上', '模型', '1', '學習', '2', '有', '訓練', '3', '與', '使用', '來', '從', '對', '數據', '我們', '需要', '通過', '為'], ['的', '和', '在', '是', '了', '個', '中', '可以', 'io', '學習', '與', '上', '一個', 'n', '我們', '到', '為', '都', '等', '而', '進行', '1', '2', '函數', '數據', '也', '訓練', '=', '以', '對'], ['的', '和', '在', 'io', '了', '是', '個', '上', '學習', '1', '我們', '為', '中', '網絡', 'n', '=', '一個', '可以', '訓練', '也', '與', '模型', '有', '使用', '年', '進行', '從', '對', '輸入', '特徵']]
itera: 71
[['的', '在', '和', '是', '了', '個', 'io', '中', '學習', '可以', '一個', '上', '為', 'n', '與', '=', '進行', '網絡', '1', '模型', '我們', '而', 'GPU', '訓練', '到', '來', '都', '有', '也', '對'], ['的',

itera: 80
[['的', '和', '在', '了', '是', '個', '中', 'io', '1', '3', '我們', '上', '也', '與', '=', '學習', '一個', '網絡', '為', '等', '到', '數據', '而', '算法', 'n', '函數', '年', '就', '可以', '深度'], ['的', '和', '在', '是', '了', 'io', '中', '個', '上', 'n', '一個', '與', '我們', '可以', '為', '也', '需要', '等', '使用', '1', '模型', '2', '學習', '有', '算法', '訓練', '就', '對', '網絡', '從'], ['的', '了', '在', '和', 'io', '是', '=', '1', '一個', '個', '我們', '中', 'n', '為', '模型', '2', '可以', '學習', '對', '有', '而', '來', '使用', '網絡', '上', '技術', '也', '方法', '算法', '那麼'], ['的', '在', '是', '和', '了', '中', '個', 'io', '=', '學習', '為', '1', '可以', '一個', '訓練', '我們', '到', '上', 'n', '層', '也', '2', '與', '來', '問題', '例如', '對', '都', '年', '會'], ['的', '在', '和', '了', '是', 'io', '個', '1', 'n', '一個', '可以', '=', '訓練', '學習', '中', '進行', '為', '模型', '有', '2', '上', 'GPU', '數據', '使用', '網絡', '以', '與', '我們', '對', '或']]
itera: 81
[['的', '在', '和', '是', '了', '個', '中', 'io', '我們', '一個', '=', '訓練', '可以', '學習', '需要', 'n', '到', '2', '層', '進行', '其', '算法', '年', '3', '與', '為', '網絡', '有', '更', '特徵'], [

itera: 90
[['的', '在', '和', '是', 'io', '了', '個', '中', '上', '=', '一個', '可以', 'n', '我們', '1', '訓練', '網絡', '為', '2', '算法', '函數', '也', '模型', '數據', '與', '進行', '來', '到', '對', 'GPU'], ['的', '在', '和', '了', '是', 'io', '中', '個', '一個', '1', '與', '學習', '我們', '2', '可以', 'n', '3', '通過', '為', '有', '也', '網絡', '就', '上', '訓練', '而', '算法', '=', '下'], ['的', '和', '在', '了', '是', '個', 'io', '一個', '1', '為', '中', '可以', '學習', '也', '與', '進行', '=', '對', '我們', 'n', '使用', '輸入', '來', '就', '訓練', '你', '模型', '到', '表示', '層'], ['的', '和', '在', '了', '是', 'io', '中', '個', '上', '學習', 'n', '為', '也', '與', '1', '模型', '訓練', '我們', '有', '等', '網絡', '而', '使用', '可以', '圖像', '需要', '數據', '對', '到', '都'], ['的', '在', '和', '=', '了', '是', 'io', '中', '1', '個', '3', '學習', '2', '一個', '為', 'n', '可以', '我們', '模型', '到', '你', '需要', '更', '對', '進行', 'x', '從', '數據', '來', 'GPU']]
itera: 91
[['的', '在', '和', '了', '是', 'io', '中', '一個', '個', '可以', 'n', '學習', '算法', '1', '與', '我們', '對', '以', '從', '也', '到', '為', '等', '=', '上', '函數', '進行', '使用', '就', '更'], ['的', '

CPU times: user 3h 48min 37s, sys: 51.5 s, total: 3h 49min 28s
Wall time: 3h 50min 32s


In [296]:
print('all powerterm')
display(pd.DataFrame(powerterm_eachiter[-1][:20]))
print('by topic')
topic_powerterm = pd.DataFrame(powerterm_list_eachiter[-1])
display(topic_powerterm.transpose())
#powerterms

all powerterm


,0,1
0,深度學習,1723.2
1,機器學習,420.4
2,卷積神經網絡,376.2
3,20,303.6
4,深度學習的,294.5
5,但是,231.8
6,這個,221.4
7,10,191.8
8,神經網絡的,182.5
9,的深度學習,172.5


by topic


,0,1,2,3,4
0,"(深度學習, 1723.2)","(學習的, 173.7)","(學習的, 173.7)","(學習的, 173.7)","(學習的, 173.7)"
1,"(學習的, 173.7)","(網絡的, 148.2)","(網絡的, 148.2)","(網絡的, 148.2)","(網絡的, 148.2)"
2,"(就是, 168.4)","(的數據, 130.5)","(的模型, 107.1)","(的數據, 130.5)","(是一個, 102.0)"
3,"(的特徵, 140.4)","(學習算法, 127.2)","(是一個, 102.0)","(學習算法, 127.2)","(中的, 75.8)"
4,"(的深度, 127.2)","(的模型, 107.1)","(12, 99.6)","(的模型, 107.1)","(的網絡, 74.7)"
5,"(是一個, 102.0)","(是一個, 102.0)","(學習模型, 79.6)","(是一個, 102.0)","(的是, 69.6)"
6,"(也就, 83.8)","(12, 99.6)","(中的, 75.8)","(12, 99.6)","(我們可以, 67.6)"
7,"(中的, 75.8)","(學習模型, 79.6)","(的網絡, 74.7)","(學習模型, 79.6)","(13, 60.4)"
8,"(的是, 69.6)","(層的, 77.2)","(的是, 69.6)","(中的, 75.8)","(從而, 58.8)"
9,"(的訓練, 69.3)","(中的, 75.8)","(模型的, 69.0)","(的網絡, 74.7)","(到的, 56.8)"


In [278]:
print('all powerterm')
display(pd.DataFrame(powerterm_eachiter[n][:20]))
print('by topic')
topic_powerterm = pd.DataFrame(powerterm_list_eachiter[n])
display(topic_powerterm.transpose())
#powerterms
'''
for i in range(args.iter):
    print('\n\niter ',i)
    topic_powerterm = pd.DataFrame(powerterm_list_eachiter[i])
    display(topic_powerterm.transpose().head()) 
    top_powerterm = pd.DataFrame(powerterm_eachiter[i])
    display(top_powerterm[:5+i])
'''

all powerterm


,0,1
0,的,1760.9
1,深度學習,1723.2
2,網絡,746.6
3,1,692.7
4,n,641.7
5,2,639.6
6,是,587.9
7,在,556.9
8,我們,490.2
9,3,441.8


by topic


,0,1,2,3,4
0,"(學習的, 173.7)","(學習的, 173.7)","(的, 1760.9)","(學習的, 173.7)","(學習的, 173.7)"
1,"(網絡的, 148.2)","(網絡的, 148.2)","(學習, 1009.2)","(就是, 168.4)","(的特徵, 140.4)"
2,"(學習算法, 127.2)","(的特徵, 140.4)","(網絡, 746.6)","(的模型, 107.1)","(的模型, 107.1)"
3,"(的模型, 107.1)","(的數據, 130.5)","(1, 692.7)","(是一個, 102.0)","(是一個, 102.0)"
4,"(是一個, 102.0)","(的模型, 107.1)","(n, 641.7)","(也就, 83.8)","(12, 99.6)"
5,"(學習模型, 79.6)","(是一個, 102.0)","(2, 639.6)","(學習模型, 79.6)","(學習模型, 79.6)"
6,"(中的, 75.8)","(12, 99.6)","(是, 587.9)","(層的, 77.2)","(中的, 75.8)"
7,"(的網絡, 74.7)","(學習模型, 79.6)","(在, 556.9)","(中的, 75.8)","(的是, 69.6)"
8,"(的是, 69.6)","(中的, 75.8)","(我們, 490.2)","(的是, 69.6)","(的訓練, 69.3)"
9,"(的訓練, 69.3)","(的網絡, 74.7)","(3, 441.8)","(的訓練, 69.3)","(模型的, 69.0)"


"\nfor i in range(args.iter):\n    print('\n\niter ',i)\n    topic_powerterm = pd.DataFrame(powerterm_list_eachiter[i])\n    display(topic_powerterm.transpose().head()) \n    top_powerterm = pd.DataFrame(powerterm_eachiter[i])\n    display(top_powerterm[:5+i])\n"

In [234]:
powerterm_eachiter[-1]

[('的', 1760.9),
 ('深度學習', 1723.2),
 ('網絡', 746.6),
 ('1', 692.7),
 ('n', 641.7),
 ('2', 639.6),
 ('是', 587.9),
 ('一', 585.1),
 ('在', 556.9),
 ('這', 541.7),
 ('我們', 490.2),
 ('神經', 480.8),
 ('模型', 446.6),
 ('數據', 444.0),
 ('3', 441.8),
 ('一個', 432.0),
 ('用', 422.1),
 ('機器學習', 420.4),
 ('人工智慧', 420.0),
 ('0', 377.7),
 ('卷積神經網絡', 376.2),
 ('訓練', 374.4),
 ('可以', 358.6),
 ('Deep', 347.6),
 ('com', 338.1),
 ('了', 335.8),
 ('有', 330.1),
 ('中', 327.6),
 ('特徵', 307.8),
 ('GPU', 306.6),
 ('20', 303.6),
 ('和', 296.7),
 ('深度學習的', 294.5),
 ('層', 287.5),
 ('如', 285.2),
 ('大', 283.2),
 ('算法', 273.8),
 ('C', 273.6),
 ('計算', 273.2),
 ('問題', 272.6),
 ('A', 271.2),
 ('4', 267.9),
 ('為', 265.0),
 ('不', 259.7),
 ('能', 257.2),
 ('例如', 257.2),
 ('作者', 246.4),
 ('輸入', 244.4),
 ('來', 243.1),
 ('方法', 241.0),
 ('S', 241.0),
 ('圖', 240.2),
 ('因此', 239.2),
 ('而', 236.5),
 ('5', 236.3),
 ('Google', 235.2),
 ('但是', 231.8),
 ('使用', 231.4),
 ('上', 229.6),
 ('對', 223.3),
 ('這個', 221.4),
 ('圖像', 221.4),
 ('如果', 216.0),
